In [10]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from scipy.optimize import minimize

In [11]:
tickers = [f"Ticker_{i}" for i in range(3)]
num_values = 10**6
annual_stock_price_df = pd.DataFrame(np.random.uniform(-10, 10, size=(num_values, len(tickers))), columns=tickers)

annual_stock_price_df.head(3)

,Ticker_0,Ticker_1,Ticker_2
0,2.238395,4.255560,-3.912431
1,3.747122,7.400302,6.812247
2,3.541459,7.777374,-8.474512


In [12]:
def create_functional_1(random_values):
    def wrapper(weights):
        weighted_annual_ratio_sum_df = random_values.mul(weights).sum(axis=1)

        weighted_expectation = weighted_annual_ratio_sum_df.mean()
        weighted_variance = weighted_annual_ratio_sum_df.var(ddof=0)
        return weighted_expectation / np.sqrt(weighted_variance)

    return wrapper


def create_functional_2(random_values):
    cov_matrix = random_values.cov(ddof=0).values
    np.fill_diagonal(cov_matrix, 0)
    
    def wrapper(weights):
        numerator = weights @ random_values.mean().values
        denominator_var = (weights ** 2) @ random_values.var(ddof=0).values
        denominator_cov = (weights @ cov_matrix) @ weights
        return numerator / np.sqrt(denominator_var + denominator_cov)

    return wrapper

In [13]:
def constraint(x):
    return sum(x) - 1

con = {"type": "eq", "fun": constraint}
bounds = [(0, 1) for _ in range(len(tickers))]

functional_1 = create_functional_1(annual_stock_price_df)
functional_2 = create_functional_2(annual_stock_price_df)

minimization_functional_1 = lambda weights: - functional_1(weights)
minimization_functional_2 = lambda weights: - functional_2(weights)

In [14]:
def generate_random_weights(vector_size):
    weights = np.random.rand(vector_size)
    weights /= np.sum(weights)
    return weights


def random_search_for_initial_weights(functional_to_minimize, n_tries, vector_size):
    best_func_value = np.inf
    for _ in tqdm(range(n_tries)):
        weights = generate_random_weights(vector_size)
        res = minimize(functional_to_minimize, weights, constraints=con, bounds=bounds)
        if res.fun < best_func_value:
            best_func_value = res.fun

    return best_func_value

### 3 columns

In [15]:
random_search_for_initial_weights(minimization_functional_1, 3, vector_size=len(tickers))

100%|██████████| 3/3 [00:10<00:00,  3.61s/it]


-0.0018637031881842433

In [16]:
random_search_for_initial_weights(minimization_functional_2, 3, vector_size=len(tickers))

100%|██████████| 3/3 [00:04<00:00,  1.50s/it]


-0.0018636864982659218

### 8 columns

In [17]:
tickers = [f"Ticker_{i}" for i in range(8)]
annual_stock_price_df = pd.DataFrame(np.random.uniform(-10, 10, size=(num_values, len(tickers))), columns=tickers)

bounds = [(0, 1) for _ in range(len(tickers))]

functional_1 = create_functional_1(annual_stock_price_df)
functional_2 = create_functional_2(annual_stock_price_df)

minimization_functional_1 = lambda weights: - functional_1(weights)
minimization_functional_2 = lambda weights: - functional_2(weights)

annual_stock_price_df.head(3)

,Ticker_0,Ticker_1,Ticker_2,Ticker_3,Ticker_4,Ticker_5,Ticker_6,Ticker_7
0,3.188598,-6.403886,-0.303393,-5.039679,-4.086160,6.067583,6.493955,0.169408
1,8.057174,1.033491,8.681959,-5.593514,4.598320,-0.787006,-9.085002,-7.902727
2,8.323714,-5.832777,-4.513762,-8.799477,9.583477,-5.754185,7.364186,2.550727


In [18]:
random_search_for_initial_weights(minimization_functional_1, 3, vector_size=len(tickers))

100%|██████████| 3/3 [00:37<00:00, 12.64s/it]


-0.0027123003368270467

In [19]:
random_search_for_initial_weights(minimization_functional_2, 3, vector_size=len(tickers))

100%|██████████| 3/3 [00:47<00:00, 15.77s/it]


-0.0027121571619153502